# detectron2 for まちカドまぞく ～訓練編～

<img src="https://user-images.githubusercontent.com/33882378/79055210-2ff0e600-7c86-11ea-93c6-8a65112f80f0.jpg">

detectron2 で独自データセット学習する方法

参考にしたホームページ: https://demura.net/deeplearning/16807.html

In [1]:
import os
import numpy as np
import json
import matplotlib.pyplot as plt
import cv2
import random

---
## 独自のデータセットを読み込んで、データセットを用意する

基本的に元になっているデータ形式をゴリゴリ自分で読み込んで、detectron2 の形式に変換していく

### VoTT Export 形式からの読み込み

In [2]:
# VoTT のエクスポートファイルや、画像が格納されているディレクトリ
BASE_DIRECTORY = './vott-json-export/'
# VoTT のエクスポートファイル名
EXPORT_FILENAME = 'Machikado-export.json'
# 訓練データに使用する割合
TRAIN_RATIO = 0.7
# 乱数シード
RANDOM_STATE = 0

* vott の場合は "tags" 格納されているカテゴリ名が格納されているのでそれを読み出せば良い

In [3]:
from collections import OrderedDict

# エクスポートファイルからカテゴリ名を調べる
with open(BASE_DIRECTORY + EXPORT_FILENAME, 'r') as f:
    json_data = json.load(f)
    
CAT_NAME2ID = OrderedDict()
CAT_ID2NAME = OrderedDict()

for i, node in enumerate(json_data['tags']):
    CAT_NAME2ID[node['name']] = i
    CAT_ID2NAME[i] = node['name']

CAT_ID2NAME

OrderedDict([(0, 'Shamiko'),
             (1, 'Gosenzo'),
             (2, 'Lilith'),
             (3, 'Momo'),
             (4, 'Mikan'),
             (5, 'Mob')])

* 所定の形式の辞書のリストと返す引数の無い関数を作れば良い
* vott のアノテーションデータは "assets" に全て格納されているので、"assets" をゴリゴリ読み込んでいく。
* マスクに必要な座標データは、"regions" に格納されている。"regions" は複数の領域データを含んでいる可能性があるので全て列挙する。
領域データがない場合もあるのでそれも処理しておく。（VoTT で閲覧のみで、アノテーションされていないデータは領域データが無い）
* 領域データの "tags" にカテゴリ名が格納されているので、１つ前のセルで読み込んだタグ情報(CAT_NAME2ID)を使用して整数のIDを振っていく。
（tags は複数のカテゴリ名が格納されている可能性がある。VoTT で複数のタグをチェック出来るので注意）

In [4]:
from detectron2.structures import BoxMode

# machikado用にアレンジした読み込み関数
def get_machikado_dicts():
    with open(BASE_DIRECTORY + EXPORT_FILENAME, 'r') as f:
        json_data = json.load(f, object_pairs_hook=OrderedDict) # データ順を固定しておく
    
    assets = json_data['assets']

    dataset_dicts = []
    for item in assets.values():
        asset = item['asset']
        regions = item['regions']

        if len(regions) == 0:
            print('警告: name: {} - 領域データが空だったのでスキップ'.format(asset['name']))
            continue

        record = {}
        record['file_name'] = BASE_DIRECTORY + asset['name']
        record['height'] = asset['size']['height']
        record['width'] = asset['size']['width']

        objs = []
        for region in regions:
            points = region['points']
            assert len(points), '座標データが無い！'

            if len(region['tags']) > 1:
                print('警告: name: {} - 複数のタグを確認！ tags: {}'.format(asset['name'], region['tags']))

            poly = []
            for pt in points:
                poly += [pt['x'], pt['y']]

            bbox = region['boundingBox']

            obj = {
                'bbox': [bbox['left'], bbox['top'], bbox['left'] + bbox['width'], bbox['top'] + bbox['height']],
                'bbox_mode': BoxMode.XYWH_ABS, # XYWH_REL はまだサポートされていないらしい
                'segmentation': [poly],
                'category_id': CAT_NAME2ID[region['tags'][0]],
                'iscrowd': 0
            }
            objs.append(obj)

        record['annotations'] = objs
        dataset_dicts.append(record)
        
    return dataset_dicts

### DatasetCatalogを用意する

* 初めから訓練、テストが分かれてれば良いが、一緒のフォルダでも分割後にラムダ式で指定すれば良い

In [5]:
from detectron2.data import DatasetCatalog

dataset_dicts = get_machikado_dicts()

# 訓練用、テスト用に分ける
random.seed(RANDOM_STATE)
random.shuffle(dataset_dicts)

split_idx = int(len(dataset_dicts) * TRAIN_RATIO) + 1

# 登録
DatasetCatalog.clear()
DatasetCatalog.register('train', lambda : dataset_dicts[:split_idx])
DatasetCatalog.register('test', lambda : dataset_dicts[split_idx:])

警告: name: 53.jpg - 領域データが空だったのでスキップ
警告: name: 52.jpg - 領域データが空だったのでスキップ
警告: name: 51.jpg - 領域データが空だったのでスキップ
警告: name: 50.jpg - 領域データが空だったのでスキップ
警告: name: 49.jpg - 領域データが空だったのでスキップ
警告: name: 48.jpg - 領域データが空だったのでスキップ


---

## 学習

* engine/defaults.py 399 行目コメントアウトでウザイ モデル表示をしないように出来る
* モデルを変更したい場合は https://github.com/facebookresearch/detectron2/blob/master/MODEL_ZOO.md ここに色々あるので試すと良い
* ```cfg.MODEL.WEIGHTS = ``` の部分は使用したい重みファイルに変える。(ダウンロードしてきた重みファイルへのパスを設定する)
* merge_from_file はきちんと推論の時も揃えないとダメ！！

In [6]:
from detectron2.config import get_cfg

cfg = get_cfg()
cfg.OUTPUT_DIR = './output'
cfg.CUDA = 'cuda:0'

# cfg.merge_from_file("../configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
# cfg.MODEL.WEIGHTS = './coco_models/model_final_f10217.pkl'
# cfg.SOLVER.IMS_PER_BATCH = 2

# 重いけど、これ精度良いです。
cfg.merge_from_file('../configs/COCO-InstanceSegmentation/mask_rcnn_X_101_32x8d_FPN_3x.yaml')
cfg.MODEL.WEIGHTS = './coco_models/model_final_2d9806.pkl'
cfg.SOLVER.IMS_PER_BATCH = 1 # GTX2070 ではこれが限界

cfg.DATASETS.TRAIN = ('train',)
cfg.DATASETS.TEST = ()   # no metrics implemented for this dataset
cfg.DATALOADER.NUM_WORKERS = 2
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.MAX_ITER = 1500    # 300 iterations seems good enough, but you can certainly train longer <- とあるが、まあデータセットによるよね・・・
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   # faster, and good enough for this toy dataset
cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(CAT_ID2NAME) 

In [7]:
# DefaultTrainer はサンプルなので、ガチにやる人は自分で作るらしい・・・
from detectron2.engine import DefaultTrainer

# 出力先のディレクトリを作る
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False) # True で途中から学習できるらしい
trainer.train()

[04/11 20:59:30 d2.data.build]: Removed 0 images with no usable annotations. 34 images left.
[04/11 20:59:30 d2.data.common]: Serializing 34 elements to byte tensors and concatenating them all ...
[04/11 20:59:30 d2.data.common]: Serialized dataset takes 0.07 MiB
[04/11 20:59:30 d2.data.detection_utils]: TransformGens used in training: [ResizeShortestEdge(short_edge_length=(640, 672, 704, 736, 768, 800), max_size=1333, sample_style='choice'), RandomFlip()]
[04/11 20:59:30 d2.data.build]: Using training sampler TrainingSampler


'roi_heads.box_predictor.cls_score.weight' has shape (81, 1024) in the checkpoint but (7, 1024) in the model! Skipped.
'roi_heads.box_predictor.cls_score.bias' has shape (81,) in the checkpoint but (7,) in the model! Skipped.
'roi_heads.box_predictor.bbox_pred.weight' has shape (320, 1024) in the checkpoint but (24, 1024) in the model! Skipped.
'roi_heads.box_predictor.bbox_pred.bias' has shape (320,) in the checkpoint but (24,) in the model! Skipped.
'roi_heads.mask_head.predictor.weight' has shape (80, 256, 1, 1) in the checkpoint but (6, 256, 1, 1) in the model! Skipped.
'roi_heads.mask_head.predictor.bias' has shape (80,) in the checkpoint but (6,) in the model! Skipped.


[04/11 20:59:31 d2.engine.train_loop]: Starting training from iteration 0
[04/11 20:59:39 d2.utils.events]:  eta: 0:08:37  iter: 19  total_loss: 3.169  loss_cls: 1.920  loss_box_reg: 0.551  loss_mask: 0.691  loss_rpn_cls: 0.009  loss_rpn_loc: 0.031  time: 0.3509  data_time: 0.0131  lr: 0.000005  max_mem: 2767M
[04/11 20:59:46 d2.utils.events]:  eta: 0:08:34  iter: 39  total_loss: 3.157  loss_cls: 1.779  loss_box_reg: 0.594  loss_mask: 0.690  loss_rpn_cls: 0.010  loss_rpn_loc: 0.048  time: 0.3496  data_time: 0.0042  lr: 0.000010  max_mem: 2767M
[04/11 20:59:53 d2.utils.events]:  eta: 0:08:34  iter: 59  total_loss: 2.880  loss_cls: 1.508  loss_box_reg: 0.652  loss_mask: 0.687  loss_rpn_cls: 0.007  loss_rpn_loc: 0.025  time: 0.3519  data_time: 0.0041  lr: 0.000015  max_mem: 2767M
[04/11 21:00:00 d2.utils.events]:  eta: 0:08:22  iter: 79  total_loss: 2.597  loss_cls: 1.200  loss_box_reg: 0.647  loss_mask: 0.683  loss_rpn_cls: 0.009  loss_rpn_loc: 0.045  time: 0.3507  data_time: 0.0044  lr: